# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Mbti_LexicalTendency/src/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Mbti_LexicalTendency/src/']


In [2]:
from tqdm import tqdm
import config as cfg
import pandas as pd
import swifter
from pshmodule.utils import filemanager as fm

In [3]:
df = fm.load('/content/drive/MyDrive/Mbti_LexicalTendency/data/NF.xlsx')

extension : .xlsx
Loaded 462 records from /content/drive/MyDrive/Mbti_LexicalTendency/data/NF.xlsx


# Check Algorithm

공유드린 곳에 들어가면 아까 보셨던 "패턴조합"이 있고요<br>
제가 찾고 싶은 것은  이 안에서<br>
"뽑혀 나온 패턴 들 중에서 '경청표시-느낌표시-부연-동의' 인 것은?"<br><br>

(조건)<br>
네 개의 패턴표지의 순서는 지켜져야 함.<br>
표지의 순서가 유지된다면, 표지 전후에 다른 표지가 끼어들거나 일부표지가 반복되어 나타나도 상관없음.<br><br>

(정답/오답 예시)<br>
예를 들어 "경청-느낌-느낌-경청-부연-동의" <- 이것은 정답이고 (순서 O, 표지 포함O)<br>
"동의-부연-경청-느낌-동의-부연-동의" <- 이것은 정답 (순서 O, 표지 포함O)<br>
"느낌-느낌-경청-부연" <- 이것은 오답 (순서X , 표지 포함X)<br>
"경청-부연-동의" <- 이것은 오답(순서 O, 표지 포함X)<br>

In [4]:
df.head()

,유형번호,패턴조합,출현빈도
0,1,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-동의 표시,4
1,2,동의 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시,3
2,3,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시,3
3,4,느낌 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시,3
4,5,경청 표시-경청 표시-경청 표시-경청 표시-반복 진술-경청 표시-경청 표시-동의 표...,3


In [5]:
df.dropna(inplace=True)

In [6]:
df.shape

(461, 3)

패턴 찾기

In [7]:
pattern = {'경청': 0, '느낌': 1, '부연': 2, '동의': 3}

In [8]:
def find_subsequence(subsequence, mainsequence):
    sub_idx = 0
    for item in mainsequence:
        if item == subsequence[sub_idx]:
            sub_idx += 1
            if sub_idx == len(subsequence):
                return True
    return False

In [13]:
result = []

for i in df.iterrows():
  check = []

  # - split 후 중복 제거  
  temp = ['']
  for p in i[1]['패턴조합'].split('-'):
    if temp[-1] != p:
      temp.append(p)
  temp.remove('')
  
  # 번호 매기기
  for t in temp:
    for s in t.split():
      if s in pattern.keys():
        check.append(pattern[s])


  # 순서대로 있는지 체크
  check_temp = find_subsequence([0, 1, 2, 3], check)
  result.append(str(check_temp))

In [14]:
len(result)

461

In [15]:
df['result'] = result

In [18]:
df[df.result=='True']

,유형번호,패턴조합,출현빈도,result
16,17,경청 표시-느낌 표시-경청 표시-경청 표시-경청 표시-부연-동의 표시,2,True
60,61,경청 표시-부연-느낌 표시-부연-후행 발화 예측-경청 표시-동의 표시-경청 표시-경...,1,True
63,64,동의 표시-동의 의견 제시-느낌 표시-경청 표시-경청 표시-느낌 표시-부연-경청 표...,1,True
64,65,동의 의견 제시-느낌 표시-경청 표시-경청 표시-느낌 표시-부연-경청 표시-부연-동...,1,True
65,66,경청 표시-경청 표시-느낌 표시-부연-경청 표시-부연-동의 표시-부연-느낌 표시,1,True
94,95,부연-화제 전개 및 발전-경청 표시-후행 발화 직접 요청-정확성 확인-느낌 표시-부...,1,True
151,152,경청 표시-경청 표시-느낌 표시-경청 표시-사실적 공감-부연-동의 표시,1,True
219,220,반복 진술-반복 진술-경청 표시-경청 표시-반복 진술-느낌 표시-느낌 표시-부연-동...,1,True
273,274,반복 진술-동의 표시-느낌 표시-느낌 표시-경청 표시-경청 표시-경청 표시-느낌 표...,1,True
274,275,동의 표시-느낌 표시-느낌 표시-경청 표시-경청 표시-경청 표시-느낌 표시-경청 표...,1,True


# Save

In [21]:
fm.save('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/NF_pattern_check.xlsx', df)

Saved 461 records
